In [1]:
#创建政策实体表
from py2neo import NodeMatcher

# 其他代码...
import pandas as pd
from py2neo import Graph, Node

# 连接到Neo4j数据库
graph = Graph('http://localhost:7474',user='neo4j',password='11111111',name = 'neo4j')

# 加载Excel文件（假设Excel文件命名为file.xlsx，表单名为Sheet1）
df = pd.read_excel('地方和中央、补充汇总_37145条_政策实体表.xlsx', sheet_name='Sheet2')
    
# 创建 NodeMatcher
matcher = NodeMatcher(graph)

# 遍历DataFrame中的每一行
for index, row in df.iterrows():
    # 首先检查节点是否已存在
    node = matcher.match("Regulation", code=row['编号']).first()
    if not node:
        node = Node("Regulation",
                    编号=row['编号'],  # 假设这里使用 “编号” 作为节点的唯一标识符
                    标题=row['标题'],
                    链接=row['链接'],
                    是否有效=row['是否有效'],
                    效力位阶=row['效力位阶'],
                    发文字号=row['发文字号'],
                    发布日期=row['发布日期'],
                    施行日期=row['施行日期'],
                    法规类别=row['法规类别'])
        graph.create(node)

In [21]:
#从实体表中拆解出发布关系和机构
import pandas as pd
import json
from ast import literal_eval
# 读取Excel文件
df = pd.read_excel('地方和中央、补充汇总_37145条.xlsx')

# 存储关系数据的列表
relations = []

# 存储发文机构的字典，key为机构名，value为编号
institutions_dict = {}

# 当前的机构编号
current_id = 1

# 遍历DataFrame中的每一行
for index, row in df.iterrows():
    code = row['编号']
    
    # 使用literal_eval代替json.loads
    institutions_string = row['发文机构']
    try:
        institutions = literal_eval(institutions_string)
    except ValueError as e:
        print(f"Error parsing institutions for code {code}: {e}")
        continue
    
    # 对于每个机构，添加到关系列表和机构字典中
    for institution in institutions:
        relations.append({'编号': code, '发文机构': institution})
        if institution not in institutions_dict:
            institutions_dict[institution] = f'J{current_id}'
            current_id += 1

# 创建关系DataFrame
relations_df = pd.DataFrame(relations)

# 添加发文机构编号列
relations_df['发文机构编号'] = relations_df['发文机构'].apply(lambda x: institutions_dict[x])

# 现在我们有了发文机构对应的编号，保存到一个新的Excel文件中
institutions_df = pd.DataFrame(list(institutions_dict.items()), columns=['发文机构', '机构编号'])
institutions_df.to_excel('institutions.xlsx', index=False)

# 保存更新后的关系表到新的Excel文件中
relations_df.to_excel('relations_with_institutions_codes.xlsx', index=False)

In [2]:
#创建机构表
from py2neo import NodeMatcher

# 其他代码...
import pandas as pd
from py2neo import Graph, Node

# 连接到Neo4j数据库
graph = Graph('http://localhost:7474',user='neo4j',password='11111111',name = 'neo4j')

# 加载Excel文件（假设Excel文件命名为file.xlsx，表单名为Sheet1）
df = pd.read_excel('发文机构.xlsx')
    
# 创建 NodeMatcher
matcher = NodeMatcher(graph)

# 遍历DataFrame中的每一行
for index, row in df.iterrows():
    # 首先检查节点是否已存在
    node = matcher.match("Institution", code=row['机构编号']).first()
    if not node:
        node = Node("Institution",
                    机构编号=row['机构编号'],  # 假设这里使用 “编号” 作为节点的唯一标识符
                    发文机构=row['发文机构'],
                    级别=row['级别'])
        graph.create(node)

In [3]:
#导入机构-政策关系表
from py2neo import Node, Relationship
from py2neo import NodeMatcher

import pandas as pd
from py2neo import Graph, Node

# 连接到Neo4j数据库
graph = Graph('http://localhost:7474',user='neo4j',password='11111111',name = 'neo4j')

# 加载Excel文件（假设Excel文件命名为file.xlsx，表单名为Sheet1）
df = pd.read_excel('发文机构_政策_关系表.xlsx')
    
# 遍历DataFrame中的每一行
for index, row in df.iterrows():
    source_node = Node("Institution", 机构编号=row["发文机构编号"])
    target_node = Node("Regulation", 编号=row["编号"])

    # 检查是否已存在相同属性的节点
    graph.merge(source_node, "Institution", "机构编号")
    graph.merge(target_node, "Regulation", "编号")

    # 创建关系
    relation = Relationship(source_node, row["关系"], target_node)

    # 将关系添加到图中
    graph.merge(relation)

In [ ]:
#导入政策之间的关系表
from py2neo import Node, Relationship
from py2neo import NodeMatcher

import pandas as pd
from py2neo import Graph, Node

# 连接到Neo4j数据库
graph = Graph('http://localhost:7474',user='neo4j',password='11111111',name = 'neo4j')

# 加载Excel文件（假设Excel文件命名为file.xlsx，表单名为Sheet1）
df = pd.read_excel('地方和中央、补充关系汇总_26419条.xlsx')
    
# 遍历DataFrame中的每一行
for index, row in df.iterrows():
    source_node = Node("Regulation", 编号=row["政策A编号"])
    target_node = Node("Regulation", 编号=row["政策B编号"])
    graph.merge(source_node, "Regulation", "编号")
    graph.merge(target_node, "Regulation", "编号")
    
    rel = row["关系"]
    if (rel == "引用"):
        relation = Relationship(source_node, row["关系"], target_node)
        print("引用" )
    elif (rel == "相似于"):
        relation = Relationship(source_node, row["关系"], target_node)
        print("相似于")
    elif (rel == "被引用"):
        relation = Relationship(target_node, "引用", source_node)
        print("被引用")
    # 将关系添加到图中
    graph.merge(relation)